In [ ]:
import openai
import json
import random
from typing import List, Tuple, Dict
import time
from collections import Counter
import re

class DirectEntityNERGenerator:
    def __init__(self, api_key: str, model: str = "gpt-4"):
        """
        직접 엔티티 형식으로 NER 데이터를 생성하는 클래스
        
        Args:
            api_key: OpenAI API 키
            model: 사용할 GPT 모델명
        """
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model
        self.results = []
        
    def create_direct_entity_prompt(self) -> str:
        """직접 엔티티 형식으로 생성하는 프롬프트"""
        
        prompt = '''당신은 한국어 제품 리뷰에서 감정과 속성을 추출하는 NER 데이터 생성 전문가입니다.

## 🎯 **데이터 증강 우선순위 (현재 부족한 클래스 집중)**

**증강 대상:**
- 브랜드#가격
- 브랜드#다양성
- 브랜드#디자인
- 브랜드#인지도
- 브랜드#편의성
- 패키지/구성품#가격
- 패키지/구성품#다양성
- 패키지/구성품#디자인
- 패키지/구성품#인지도
- 패키지/구성품#편의성

## 📋 **엔티티 유형과 속성**

**엔티티 유형:**
- **브랜드**: 모든 분야의 제조사/브랜드 (전자, 패션, 화장품, 식품, 자동차, 가구 등 제한 없음)
- **패키지/구성품**: 포장재, 액세서리, 부속품, 사은품 등

**속성:**
- **가격**: 비용, 가성비, 할인, 경제성
- **다양성**: 선택의 폭, 옵션, 변형, 종류
- **인지도**: 유명함, 인기, 브랜드 파워, 화제성
- **편의성**: 사용 편리함, 접근성, 사용성
- **디자인**: 외관, 색상, 스타일, 미적 요소, 시각적 매력

## **생성 전략**

1. **다양한 분야의 브랜드, 패키지/구성품 활용**: 전자제품뿐만 아니라 화장품, 패션, 식품, 자동차, 가구, 문구, 완구, 스포츠용품 등 모든 분야
2. **실제 존재하는 브랜드명, 패키지/구성품명 사용**: 국내외 유명 브랜드들을 제한 없이 활용, 패키지/구성품도 제한 없이 활용
3. **다양한 스타일**: 리뷰, 설명, SNS 포스팅 등 다양한 스타일로 문장 생성

## **중요 규칙**

1. **실제 브랜드명 및 패키지/구성품 명 사용** - 추상적 표현 금지
2. **조사/어미 제외** - 핵심 명사/명사구만 추출
3. **다양한 분야 포괄** - 특정 카테고리에 국한되지 않음
4. **높은 다양성** - 최대한 다양한 브랜드/구성품 활용, 실제 있는 중소기업들 혹은 그들의 물품도 활용
5. **positive/neutral/negative의 적절한 분배** - 각 속성마다 positive/neutral/negative 문장의 수가 **각각 0.5:2:2 비율**로 나올 수 있도록 설정

## **출력 형식**

문장: [생성된 리뷰 문장]
엔티티: [["엔티티타입#속성", [["단어", 시작위치, 끝위치], ...], "감정"], ...]

**참고사항**: 
- 감정: "positive", "negative", "neutral" 중 선택
- 엔티티에 해당하는 명확한 단어가 없으면 [None, 0, 0] 사용
- 시작위치와 끝위치는 문자 단위 인덱스 (0부터 시작)

## **기존 데이터 예시 (참고용)**

문장: 필립스같은 유명 브랜드는 아니지만 이슈가 된 전자파테스트도 통과했다네요.
엔티티: [["브랜드#인지도", [null, 0, 0], "negative"]]

문장: 본래도 아윤채가 뷰티 유투버나 SNS에서도 핫해서 써보고 싶었거든영 ㅎ
엔티티: [["브랜드#인지도", ["아윤채", 4, 7], "positive"]]

문장: 좋아하는 코스메틱 브랜드 중 하나인 #셀라피 ..💜
엔티티: [["브랜드#일반", ["셀라피", 21, 24], "positive"]]

문장: &name&씨가 이날 장착한 패딩과 신발은 모두 #오니즈카타이거 제품이었는데요-
엔티티: [["브랜드#일반", ["오니즈카타이거", 28, 35], "positive"]]

문장: 역시 #브이티코스메틱 😘
엔티티: [["브랜드#일반", ["브이티코스메틱", 4, 11], "positive"]]

문장: 신생아부터 성인까지 안심하고 사용하는 #믿고쓰는몽디에스
엔티티: [["브랜드#일반", ["몽디에스", 26, 30], "positive"]]

문장: 4가지 패키지를 내맘대로 골라쓰는 재미까지!
엔티티: [["패키지/구성품#다양성", ["4가지 패키지", 0, 7], "positive"]]

문장: 제룰 좋아하는 노랑노랑 컬러에 디자인도 쪼꼬미라 운동 나갈 때 주머니에도 쏙~
엔티티: [["패키지/구성품#디자인", ["노랑노랑 컬러", 8, 15], "positive"], ["제품 전체#디자인", [null, 0, 0], "positive"]]

문장: 그리고 쓸데없이 비싼 부가 액세서리.
엔티티: [["패키지/구성품#가격", ["부가 액세서리", 12, 19], "negative"]]

문장: 귀여운것은 덤~~^^
엔티티: [["패키지/구성품#디자인", [null, 0, 0], "positive"]]

문장: 넘 예쁜 # #오드오랄 치약 #취향저격!
엔티티: [["패키지/구성품#디자인", ["오드오랄 치약", 8, 15], "positive"]]

문장: 패키지도 고급지고 예뻐서 #출산선물 로도 굿굿❗🤘
엔티티: [["패키지/구성품#디자인", ["패키지", 0, 3], "positive"]]

문장: 하, 포장도 이뻐요.
엔티티: [["패키지/구성품#디자인", ["포장", 3, 5], "positive"]]

문장: 물티슈는 #대디베이비 만 사용하는데 휴대용 사이즈 요거 외출할 때 진짜 편해요~ ㅋㅋ
엔티티: [["패키지/구성품#편의성", ["휴대용 사이즈", 20, 27], "positive"]]

** 지금 하나의 문장을 생성하고, 해당 문장에서 추출한 엔티티들을 다음 형식으로 출력하세요.**

문장: [자연스러운 한국어 제품 리뷰 문장]
엔티티: [["본품#편의성", [null, 0, 0], "positive"]]

반드시 이 형식을 지키고, 문장은 하나만 출력하세요.
※ 특정 브랜드(삼성, 애플 등)만 반복되지 않도록 **다양한 패키지/구성품과 브랜드**를 분산하여 사용하세요. **더 다양성을 높이세요(생전 처음보는 실제 브랜드/상품도 가능)!**
※ 감정 비율 반드시 준수: 전체 생성 문장의 감정은 반드시 **positive:neutral:negative = 0.5:2:2** 비율을 맞춰야 합니다.
'''
        
        return prompt

    def generate_data(self, num_samples: int = 10) -> List[Dict]:
        """프롬프트를 사용해 NER 데이터 생성"""
        results = []
        base_prompt = self.create_direct_entity_prompt()
        
        print(f"총 {num_samples}개의 데이터를 생성합니다...")
        
        for i in range(num_samples):
            try:
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": "당신은 한국어 NER 엔티티 데이터 생성 전문가입니다. 희귀 엔티티가 포함된 자연스러운 문장을 생성해야 합니다."},
                        {"role": "user", "content": base_prompt}
                    ],
                    temperature=0.8,
                    max_tokens=600
                )
                
                content = response.choices[0].message.content
                parsed_result = self.parse_direct_response(content)
                
                if parsed_result:
                    results.append(parsed_result)
                    self.results.append(parsed_result)
                    print(f"생성 완료 {i+1}/{num_samples}: {parsed_result['sentence']}")
                else:
                    print(f"파싱 실패 {i+1}/{num_samples}")
                
                time.sleep(1)
                
            except Exception as e:
                print(f"API 호출 실패 {i+1}/{num_samples}: {e}")
                continue
                
        return results

    def parse_direct_response(self, response: str) -> Dict:
        """GPT 응답을 파싱하여 엔티티 형식으로 변환"""
        lines = response.strip().split('\n')
        sentence = ""
        entities = []
        
        for line in lines:
            line = line.strip()
            if line.startswith('문장:'):
                sentence = line.replace('문장:', '').strip()
            elif line.startswith('엔티티:'):
                entities_str = line.replace('엔티티:', '').strip()
                try:
                    entities = eval(entities_str)
                except:
                    # 파싱 실패시 빈 리스트
                    entities = []
        
        # 엔티티가 없으면 None 추가
        if not entities:
            entities = [["None", [["None", 0, 0]], "neutral"]]
        
        # 검증 및 형식 확인
        if sentence and entities:
            return {
                'sentence': sentence,
                'entities': entities
            }
        else:
            return None

    def save_entity_data(self, data: List[Dict], filename: str = "direct_entity_data.json"):
        """엔티티 데이터 저장"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"엔티티 데이터가 {filename}에 저장되었습니다.")

    def print_examples(self, data: List[Dict], num_examples: int = 5):
        """생성된 데이터 예시 출력"""
        print(f"\n=== 생성된 데이터 예시 ({min(num_examples, len(data))}개) ===")
        for i, item in enumerate(data[:num_examples]):
            print(f"\n{i+1}. 문장: {item['sentence']}")
            print(f"   엔티티:")
            for entity in item['entities']:
                if len(entity) >= 3:
                    entity_name, words_info, sentiment = entity[0], entity[1], entity[2]
                    print(f'     ["{entity_name}", {words_info}, "{sentiment}"]')

    def add_none_entities(self, data: List[Dict]) -> List[Dict]:
        """엔티티가 없는 경우 None 엔티티 추가"""
        for item in data:
            if not item['entities'] or len(item['entities']) == 0:
                item['entities'] = [["None", [["None", 0, 0]], "neutral"]]
        return data


# 사용 예시
if __name__ == "__main__":
    API_KEY = ""
    
    # 생성기 초기화
    generator = DirectEntityNERGenerator(API_KEY, model="gpt-4o")
    
    # 데이터 생성
    print("=== 직접 엔티티 형식 NER 데이터셋 생성 ===")
    data = generator.generate_data(num_samples=1000)  # 원하는 샘플 수로 변경
    
    # None 엔티티 처리
    data = generator.add_none_entities(data)
    
    # 생성된 데이터 예시 출력
    generator.print_examples(data)
    
    # JSON 파일로 저장
    generator.save_entity_data(data, "direct_entity_ner_data2000_3.json")
    
    print("\n=== 생성 완료 ===")
    print("direct_entity_ner_data2000_3.json - 직접 엔티티 형식 데이터")
    
    # 원하는 형식 예시 출력
    print("\n=== 원하는 형식 예시 ===")
    if data:
        example = data[0]
        print(f"문장: {example['sentence']}")
        for entity in example['entities']:
            print(f'{entity}')

=== 직접 엔티티 형식 NER 데이터셋 생성 ===
총 1000개의 데이터를 생성합니다...
생성 완료 1/1000: 최근에 구입한 트라움 폼 매트리스, 디자인도 깔끔하고 다양한 옵션 선택이 가능해서 맘에 들어요.
생성 완료 2/1000: 라미 만년필의 포장은 정말 고급스러워서 선물용으로도 손색없어요.
생성 완료 3/1000: 최근에 구매한 리바이스 청바지의 디자인이 정말 독특하고 멋스러워요. 게다가 여러 종류의 스타일이 있어서 선택의 폭이 넓어요.
생성 완료 4/1000: 나이키 브랜드의 다양한 운동화 옵션이 너무 많아서 선택이 힘들었어요.
생성 완료 5/1000: 이케아의 다양한 가구 옵션은 정말 매력적이지만, 품절이 잦아서 아쉬워요.
생성 완료 6/1000: 요즘 SNS에서 핫하다는 스토케의 유모차는 디자인도 예쁘고 사용이 편해서 많은 엄마들 사이에서 인기랍니다!
생성 완료 7/1000: 바나나우유의 포장 디자인이 너무 귀엽고 가지고 다니기 편리해서 하루 종일 기분이 좋더라고요!
생성 완료 8/1000: 먼슬리랩의 패키지 디자인은 깔끔하지만 특별한 매력이 없어 보여 아쉽네요.
생성 완료 9/1000: 새로 나온 얼티밋 이어즈의 블루투스 스피커는 가성비가 정말 뛰어나네요, 디자인도 세련되고 만족스러워요!
생성 완료 10/1000: 에르메스의 다양한 액세서리는 그저 멋지다는 말밖에 안 나올 정도지만, 가격은 솔직히 너무 부담스럽네요.
생성 완료 11/1000: "아디다스 스포츠백은 다양한 디자인 옵션이 있어서 선택의 폭이 넓어요."
생성 완료 12/1000: 이케아의 다양한 수납 박스는 정말 인기가 많지만, 가격이 조금 비싼 편이에요.
생성 완료 13/1000: 코즈모나의 가격은 조금 부담스럽지만, 다양한 제품 옵션이 있어서 선택의 폭이 넓어요.
생성 완료 14/1000: 스파오의 가격은 저렴한데 디자인이 정말 세련됐어요.
생성 완료 15/1000: 쿠첸의 전기밥솥은 디자인이 심플하고 세련되어 주방의 분위기를 확 바꿔줘요.
생성 완료 16/1000: 비록 

In [3]:
import pandas as pd
augmented_df = pd.read_json("direct_entity_ner_data2000_3.json")

In [5]:
def contains_none(x):
    # 리스트 안에 None 또는 "None"이 있는지 재귀적으로 확인
    if isinstance(x, list):
        return any(contains_none(i) for i in x)
    return x is None or x == "None"

# 필터링
augmented_df = augmented_df[~augmented_df['entities'].apply(contains_none)]


In [6]:
augmented_df['id'] = augmented_df.index
augmented_df = augmented_df.rename(columns={'sentence':'sentence_form', 'entities' : 'annotation'})

C:\Users\leopp\AppData\Local\Temp\ipykernel_15024\1245537395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augmented_df['id'] = augmented_df.index


In [7]:
original_df = pd.read_csv("augmented_train_expanded_2.csv", index_col=0)

In [8]:
augmented = pd.concat([original_df, augmented_df], axis = 0)

In [ ]:
augmented.to_csv("augmented_train_expanded_3.csv")

In [61]:
import pandas as pd
df = pd.read_csv("augmented_train_expanded_3.csv", index_col=0)
df

,id,sentence_form,annotation
0,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악.,"[['본품#품질', ['기어', 16, 18], 'negative']]"
1,nikluge-sa-2022-train-00002,"이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확...","[['본품#품질', ['기어 텐션', 67, 72], 'negative']]"
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,"[['제품 전체#일반', [None, 0, 0], 'positive']]"
3,nikluge-sa-2022-train-00004,샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이...,"[['제품 전체#일반', ['샥이 없는 모델', 0, 8], 'neutral']]"
4,nikluge-sa-2022-train-00005,안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다.,"[['본품#일반', ['안장', 0, 2], 'negative']]"
...,...,...,...
993,993,최근에 구입한 마모트의 등산 가방은 다양한 색상 옵션이 있어서 정말 만족스럽습니다.,"[['브랜드#다양성', ['마모트', 11, 14], 'positive']]"
994,994,최근에 구매한 노르딕네스트의 주방용품 세트는 여전히 비싼 가격이라 아쉽네요.,"[['브랜드#가격', ['노르딕네스트', 8, 14], 'negative']]"
995,995,아소스의 신상 가방은 색상이 예쁜데 가격이 조금 높은 것 같아요.,"[['브랜드#디자인', ['아소스', 0, 3], 'positive'], ['브랜드..."
996,996,"""스위스 밀리터리의 다양한 여행가방 옵션은 어느 상황에서도 멋지게 어울려요.""","[['브랜드#다양성', ['스위스 밀리터리', 0, 8], 'positive']]"


In [62]:
def contains_none_in_len1_lists(x):
    if isinstance(x, list):
        # 현재 리스트가 길이 1이고 그 안에 None/"None"이 있는 경우
        if len(x) == 1:
            val = x[0]
            if not isinstance(val, list):
                return val is None or val == "None"
            else:
                return val[1][0] is None or val[1][0] == "None"
        else:
            # 길이가 2 이상이면 내부 요소들을 재귀적으로 검사
            return False
    else:
        # 리스트가 아니면 False (탐색 종료)
        return False

import ast
df.annotation = df.annotation.apply(ast.literal_eval)
df_none = df[df['annotation'].apply(contains_none_in_len1_lists)]
df = df[~df['annotation'].apply(contains_none_in_len1_lists)]

In [64]:
sampled_df = df_none.sample(frac=0.1, random_state=42)
n_df = pd.concat([df, sampled_df], axis=0).reset_index(drop=True)

In [66]:
n_df.to_csv("augmented_train_expanded_3(nullX_10%).csv")

In [20]:
df.to_csv("augmented_train_expanded_3.csv")